In [34]:
from scipy import optimize
import numpy as np

p = 50.0 # bar
temp = 273.15 + 220. # K
r = 8.314 # J/(mol K)

n0co = 50.
n0h2 = 100.
n0co2 = 10.
n0h2o = 0.
n0ch3oh = 0.

delta_h_1_298 = (-(-110.53) -2*0 + -200.94)*1000
delta_h_2_298 = (-(-393.50) -3*0 + -200.94 + -241.83)*1000
delta_h_3_298 = (-(-110.53) -(-241.83) + -393.50 + 0)*1000

delta_g_1_298 = (-(-137.20) -2*0 + -162.32)*1000
delta_g_2_298 = (-(-394.36) -3*0 + -162.32 + -228.62)*1000
delta_g_3_298 = (-(-137.20) -(-228.62) + -394.36 + 0)*1000

print delta_h_1_298
print delta_h_2_298
print delta_h_3_298

k1_298 = np.exp(-delta_g_1_298/(r*temp))
k2_298 = np.exp(-delta_g_2_298/(r*temp))
k3_298 = np.exp(-delta_g_3_298/(r*temp))

# Angenommen, dass Delta_H fast im Interval zwischen 298.15K und 443.15K 
# unabhängig von der Temperatur sei...
k1_443 = k1_298 * np.exp(-delta_h_1_298/8.314*( 1/443.15 - 1/298.15 ))
k2_443 = k2_298 * np.exp(-delta_h_2_298/8.314*( 1/443.15 - 1/298.15 ))
k3_443 = k3_298 * np.exp(-delta_h_3_298/8.314*( 1/443.15 - 1/298.15 ))

for f in [k1_443, k2_443, k3_443]:
    print f

def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nco2 = x_vec[2]
    nh2o = x_vec[3]
    nch3oh = x_vec[4]
    xi1 = x_vec[5]
    xi2 = x_vec[6]
    xi3 = x_vec[7]
    
    f1 = -nco + n0co - xi1 +0 -xi3
    f2 = -nh2  + n0h2 -2*xi1 -3*xi2 +xi3
    f3 = -nco2 + n0co2 +0 -xi2 +xi3
    f4 = -nh2o + n0h2o +0 +xi2 -xi3
    f5 = -nch3oh + n0ch3oh +xi1 +xi2 -0
    f6 = -k1_443 + nch3oh / (nco * nh2) * (p/1.)**-2
    f7 = -k2_443 + nch3oh * nh2o / (nco2 * nh2**2) * (p/1.)**-2
    f8 = -k3_443 + nco2 * nh2 / (nco * nh2o) * (p/1.)**0
    
    return [f1, f2, f3, f4, f5, f6, f7, f8]

sol = optimize.root(fun, np.ones([8,1])*0.01)
sol.x

-90410.0
-49270.0
-41140.0
0.00300546664952
0.000650510159013
4.62016865974


array([  1.67442275e+01,   1.72453098e+01,   4.88351580e+00,
         5.20043771e+00,   3.86733404e+01,  -2.36009010e+05,
         2.36047701e+05,   2.36042422e+05])